In [1]:
using Knet,CUDA,ArgParse, DataStructures


In [275]:
atype = CUDA.functional() ? KnetArray{Float32} : Array{Float32}

KnetArray{Float32,N} where N

In [284]:
module VGG
using Knet,CUDA,ArgParse, DataStructures
include(Knet.dir("data","imagenet.jl"))

#const vggurl = "http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-16.mat"
#check vgg.jl for original version
const LAYER_TYPES = ["conv", "relu", "pool", "fc", "prob"]

function main()
    args = Dict()
    
    args["model"] = "imagenet-vgg-verydeep-16"
    args["top"] = 5
    args["atype"] = CUDA.functional() ? KnetArray{Float32} : Array{Float32}
    
    atype = CUDA.functional() ? KnetArray{Float32} : Array{Float32}

    vgg = matconvnet(args["model"])
    (params),ldict = get_params(vgg, atype)
    
    description = vgg["meta"]["classes"]["description"]
    averageImage = convert(Array{Float32},vgg["meta"]["normalization"]["averageImage"])

    
    return vgg, params, description, averageImage, ldict
end

# This procedure makes pretrained MatConvNet VGG parameters convenient for Knet
# Also, if you want to extract features, specify the last layer you want to use
function get_params(CNN, atype; last_layer="prob")
    layers = CNN["layers"]
    weights, operations, derivatives = [], [], []
    
    
    ldict = OrderedDict()
    for l in layers
        get_layer_type(x) = startswith(l["name"], x)
        operation = filter(x -> get_layer_type(x), LAYER_TYPES)[1]
        push!(operations, operation)
        push!(derivatives, haskey(l, "weights") && length(l["weights"]) != 0)

        if derivatives[end]
            w = copy(l["weights"])
            if operation == "conv"
                w[2] = reshape(w[2], (1,1,length(w[2]),1))
            elseif operation == "fc"
                w[1] = transpose(mat(w[1]))
            end
            push!(weights, w)
            ldict[l["name"]] = w
        end

        last_layer != nothing && get_layer_type(last_layer) && break
    end

    (map(w -> map(wi->convert(atype,wi), w), weights), operations, derivatives), ldict
end


end

Main.VGG

In [285]:
vgg, params, description, averageImage, ld = VGG.main();

┌ Info: Loading imagenet-vgg-verydeep-16.mat...
└ @ Main.VGG /kuacc/users/ckoksal20/.julia/packages/Knet/OYNCT/data/imagenet.jl:12


In [5]:
keys(ld)

Base.KeySet for a OrderedDict{Any,Any} with 16 entries. Keys:
  "conv1_1"
  "conv1_2"
  "conv2_1"
  "conv2_2"
  "conv3_1"
  "conv3_2"
  "conv3_3"
  "conv4_1"
  "conv4_2"
  "conv4_3"
  "conv5_1"
  "conv5_2"
  "conv5_3"
  "fc6"
  "fc7"
  "fc8"

In [6]:
layers = [l["name"] for l in vgg["layers"]]

1×37 Array{String,2}:
 "conv1_1"  "relu1_1"  "conv1_2"  "relu1_2"  …  "fc7"  "relu7"  "fc8"  "prob"

In [7]:
#size(ld["conv1_2"][1])

(3, 3, 64, 64)

In [10]:
# Define convolutional layer --> Trainable:
struct Conv
w 
b
f
pad
ws
conv_strd
end
Conv(w1, w2, nx, ny; f=sigm,pad=0,ws=2, conv_s =1) = Conv(param(w1,w2,nx,ny), param0(1,1,ny,1),f, pad, ws, conv_s)
(c::Conv)(x) = c.f.(pool(conv4(c.w, x,padding=c.pad, stride = c.conv_strd) .+ c.b,window=c.ws))

In [11]:
# Define dense layer:
struct Dense; w; b; f; end
Dense(i,o; f=identity) = Dense(param(o,i;atype=atype), param0(o,atype=atype), f)
(d::Dense)(x) = d.f.(d.w * mat(x) .+ d.b)

In [12]:
struct Chain; layers; end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(data::Knet.Train20.Data) = mean( c(x,y) for (x,y) in data)

In [13]:
# VGG convolutions --> non trainable
struct Conv_vgg 
w
b
f
pad
p_window_size
ps # pool_stride
dil
pool_pad
end
Conv_vgg(w; f=sigm,pad=1,ws=2,ps=ws,dil=1,pool_p=0)  = Conv_vgg(atype(w[1]),atype(w[2]),f,pad,ws,ps,dil,pool_p)
Conv_vgg(w,b; f=sigm,pad=1,ws=2,ps=ws,dil=1,pool_p=0)  = Conv_vgg(atype(w),atype(b),f,pad,ws,ps,dil,pool_p)
(c::Conv_vgg)(x) = c.f.(pool(conv4(c.w, x,padding=c.pad,mode=1,dilation = c.dil) .+ c.b, window = c.p_window_size, padding=c.pool_pad,stride=c.ps));

struct Dense_vgg
w
b
f

end
Dense_vgg(w; f=sigm)  = Dense_vgg(atype(w[1]),atype(w[2]),f)
(d::Dense_vgg)(x) = d.f.(  d.w * mat(x) + d.b);

In [104]:
struct PoolLayer
mode
end
(p::PoolLayer)(x) = pool(x,mode=0)

LoadError: invalid redefinition of constant PoolLayer

In [14]:
function decimate(layer, m, atype )
    
    # Downsample by keeping every m'th value
    # Used Converting Fully Connected layer to Convolutional Layer
     
    layer = Array(layer)
    for d in 1:ndims(layer)
        println(d)
        if m[d] != nothing
            layer = selectdim(layer,d, 1:m[d]:size(layer)[d])
            #println(size(layer))
        end
    end
    return atype(layer)
end
            
    
    

decimate (generic function with 1 method)

In [16]:
function denseToConv(dense_vgg_layer::Dense_vgg)
    
    # Authors converts FC6 FC7 layers to CONV layers ;
    w = dense_vgg_layer.w
    b = dense_vgg_layer.b
    println("Before b size",size(b))
    
    (n_output,n_input) = size(w)
    
    
    if size(w) == (4096,25088) #FC()
        #new_weights  = reshape(dense_vgg_layer.w,(n_output,512,7,7))  
        
        
        new_weights = permutedims(reshape(dense_vgg_layer.w, (4096,512,7,7)),(3,4,2,1)) # shape (7,7,512,4096) 
        new_weights = decimate(new_weights, [3,3, nothing,4],atype )
        
        
        new_bias = decimate(b,[4,nothing],atype)
        new_bias = atype(reshape(Array(new_bias),(1,1,size(new_bias)[1],1)))
        return Conv_vgg(new_weights,new_bias,pad=6,dil=6,ws=1)
        
        
    elseif size(w) == (4096,4096)
        new_weights = permutedims(reshape(dense_vgg_layer.w,(4096,4096,1,1)),(4,3,2,1))  # (1,1,4096,4096)
        new_weights = decimate(new_weights, [nothing,nothing, 4,4],atype ) 
        new_bias = decimate(b, [4,nothing],atype)
        new_bias = atype(reshape(Array(new_bias),(1,1,size(new_bias)[1],1)))
        

    end
    #new_bias = reshape(new_bias, (1,1,size(new_bias)[1]),1 )
    
    #println("Before w size",size(w))
    #println("After w size : ",size(new_weights))
    #println("Before b size",size(b))
    #println("After b size : ",size(new_bias))
    
    
    # reshape Dense to 
    return Conv_vgg(new_weights,new_bias,ws=1,pad=0)
    
end

denseToConv (generic function with 1 method)

In [83]:
# Extracting results of conv4_3 and conv7 layers
function vgg_semiLayerRepresentation(x, c::Chain)
    index = 1
    conv_4_3_output = []
    #ck = KnetArray(ck)
    for l in c.layers
        x = l(x)
        if index == 10  #Conv4_3_layer_representation
            #println(typeof(x))
            conv_4_3_output =x
        end
        index +=1
    end
    
    conv_7_output = x
    return (conv_4_3_output, conv_7_output)
end

vgg_semiLayerRepresentation (generic function with 1 method)

In [108]:
vgg16 = Chain((
        #"conv1_1 - relu1_1" 
        Conv_vgg(vec(ld["conv1_1"]), f=relu ,ws=1),
        
        #"conv1_2 - relu1_2 pool1"
        Conv_vgg(vec(ld["conv1_2"]), f=relu ,ws=2),
        
        # "conv2_1"  "relu2_1"
        Conv_vgg(vec(ld["conv2_1"]), f=relu ,ws=1),
        
        #conv2_2"  "relu2_2"  "pool2"
        Conv_vgg(vec(ld["conv2_2"]), f=relu ,ws=2),
        
        #"conv3_1"  "relu3_1"
        Conv_vgg(vec(ld["conv3_1"]), f=relu ,ws=1),
        
        #"conv3_2"  "relu3_2
        Conv_vgg(vec(ld["conv3_2"]), f=relu ,ws=1),
        
        #"conv3_3"  "relu3_3"  "pool3"  #
        Conv_vgg(vec(ld["conv3_3"]), f=relu ,ws=2,pool_p=1 ),
        
        #"conv4_1"  "relu4_1"
        Conv_vgg(vec(ld["conv4_1"]), f=relu ,ws=1),
        
        #"conv4_2"  "relu4_2"
        Conv_vgg(vec(ld["conv4_2"]), f=relu ,ws=1),
        
        #"conv4_3"  "relu4_3" pool4"
        Conv_vgg(vec(ld["conv4_3"]), f=relu ,ws=1),
        PoolLayer(0), 
        
        #"conv5_1"  "relu5_1"
        Conv_vgg(vec(ld["conv5_1"]), f=relu ,ws=1),
        
        # "conv5_2"  "relu5_2"
        Conv_vgg(vec(ld["conv5_2"]), f=relu ,ws=1),
        
        #"conv5_3"  "relu5_3"  "pool5" 
        Conv_vgg(vec(ld["conv5_3"]), f=relu ,ws=3,ps=1,pool_p=1), #Poolstride 1 not changing dim
 
        #conv
        denseToConv(Dense_vgg(vec(ld["fc6"]) ,f=relu )),   #Conv_vgg(K32(3,3,512,1024) K32(1,1,1024,1) Knet.Ops20.sigm, 1, 2, 2, 1)
        #"fc6"  "relu6"  "fc8"  "prob"
          
        denseToConv(Dense_vgg(vec(ld["fc7"]) ,f=relu)),
        ));


Before b size(4096, 1)
1
2
3
4
1
2
Before b size(4096, 1)
1
2
3
4
1
2


In [109]:
# Controling model output
#x = atype(randn(Float32,(300,300,3,1)));
#k = vgg16(x);
#k


19×19×1024×1 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.615444  0.615537  0.615509  0.615486  …  0.614834  0.614829  0.614767
 0.615785  0.615912  0.615864  0.615856     0.615166  0.615166  0.615085
 0.61582   0.615966  0.615939  0.615986     0.615113  0.615128  0.615047
 0.615778  0.615961  0.615927  0.616012     0.61507   0.615104  0.614979
 0.615682  0.615938  0.615946  0.616041     0.614997  0.615033  0.614885
 0.615741  0.61595   0.615942  0.616001  …  0.614946  0.614951  0.614845
 0.616349  0.616537  0.616597  0.616554     0.615779  0.615789  0.615651
 0.616636  0.616755  0.616733  0.616658     0.616011  0.616077  0.615948
 0.61678   0.616878  0.616779  0.616708     0.616134  0.616235  0.616156
 0.616982  0.617085  0.61696   0.616924     0.616223  0.616295  0.616246
 0.616952  0.61711   0.616995  0.61695   …  0.616182  0.61618   0.616137
 0.617065  0.617187  0.617021  0.616953     0.616071  0.61622   0.616219
 0.616806  0.617045  0.616972  0.61688      0.61582   0.615988  0.616092
 

In [87]:
conv_5_3_feats,conv7_feats = vgg_semiLayerRepresentation(x,vgg16)

KnetArray{Float32,4}


(K32(19,19,512,1)[2.1879244⋯], K32(19,19,1024,1)[0.61555237⋯])

In [110]:
# Extra layers that are added to the VGG16 ---> SSD Heads
function HighOrderFeatures(conv7_feats)
    
    #conv7_feats --> (19*19*1024)
    
    conv8_1 = Conv(1,1,1024,256,  pad=0, ws=1, f=relu);
    conv8_2 = Conv(3,3, 256,512,  pad=1, conv_s=2, ws=1, f=relu);  # 10*10*512*1
    conv9_1  = Conv(1,1, 512,128, pad=0, ws=1, f=relu);  # dim. reduction because stride > 1
    conv9_2 = Conv(3,3, 128,256,  pad=1, conv_s=2, ws=1, f=relu);  # dim. reduction because stride > 1
    
    conv10_1 = Conv(1,1,256,128,pad=0,ws=1,f=relu);
    conv10_2 = Conv(3,3,128,256, pad=0,ws=1, f=relu);
    conv11_1 = Conv(1,1,256,128,ws=1, pad=0,f=relu);
    conv11_2 = Conv(3,3,128,256,ws=1, pad=0,f=relu);
    
    
    out = conv8_1(conv7_feats)
    out = conv8_2(out)
    conv8_2_feats = out
    
    out = conv9_1(out)
    out = conv9_2(out)
    conv9_2_feats = out
    
    
    out = conv10_1(out)
    out = conv10_2(out)
    conv10_2_feats = out
    
    out = conv11_1(out)
    out = conv11_2(out)
    conv11_2_feats = out
    
    return conv8_2_feats, conv9_2_feats, conv10_2_feats, conv11_2_feats
    #return out
    
end

HighOrderFeatures (generic function with 1 method)

In [111]:
conv8_2_feats, conv9_2_feats, conv10_2_feats, conv11_2_feats  =HighOrderFeatures(conv7_feats)

(K32(10,10,512,1)[0.0⋯], K32(5,5,256,1)[0.0⋯], K32(3,3,256,1)[0.0⋯], K32(1,1,256,1)[0.008990341⋯])

In [168]:
# Converting to the outputs to the number of SSD predefined filter number 8372 
function expectedShape(layerlength, batch_size, expected_output)
    
    return Integer(layerlength/batch_size/expected_output),expected_output,batch_size
    
end

expectedShape (generic function with 1 method)

In [212]:
function boundingBox_Location_Class_Predictions(x,vgg16)
    
    n_boxes = Dict(
        "conv4_3" => 4,
        "conv7" => 6,
        "conv8_2" => 6,
        "conv9_2" => 6,
        "conv10_2" => 4,
        "conv11_2" => 4)
    
    n_classes = 20 +1 # background
    
    batch_size = size(x)[end]
    #println("Batch size : ",batch_size)
    
    #Lower dimension representations
    conv4_3_feats,conv7_feats = vgg_semiLayerRepresentation(x,vgg16)#(19,19,512,1) #(19, 19, 1024, 1)
    
    #println("Size conv4_3_feats : ",size(conv4_3_feats))
    #println("Size conv7_feats : ",size(conv7_feats))
    
    
    conv8_2_feats, conv9_2_feats, conv10_2_feats, conv11_2_feats = HighOrderFeatures(conv7_feats)
    
    
    # Localization prediction convolutions (predict offsets w.r.t prior-boxes)
    loc_conv4_3 = Conv(3,3,512, n_boxes["conv4_3"] * 4,  ws=1, pad=1)
    
    loc_conv7 = Conv(3,3,1024, n_boxes["conv7"] * 4, ws=1,pad=1)
    loc_conv8_2 = Conv(3,3,512, n_boxes["conv8_2"] * 4, ws=1, pad=1)
    loc_conv9_2 = Conv(3,3,256, n_boxes["conv9_2"] * 4, ws=1, pad=1)
    loc_conv10_2 = Conv(3,3,256, n_boxes["conv10_2"] * 4, ws=1, pad=1)
    loc_conv11_2 = Conv(3,3,256, n_boxes["conv11_2"] * 4, ws=1, pad=1)

    # Class prediction convolutions (predict classes in localization boxes)
    cl_conv4_3 = Conv(3,3,512, n_boxes["conv4_3"] * n_classes,  ws=1,pad=1)
    
    cl_conv7 = Conv(3,3,1024, n_boxes["conv7"] * n_classes,  ws=1,pad=1)
    cl_conv8_2 = Conv(3,3,512, n_boxes["conv8_2"] * n_classes, ws=1, pad=1)
    cl_conv9_2 = Conv(3,3,256, n_boxes["conv9_2"] * n_classes,  ws=1, pad=1)
    cl_conv10_2 = Conv(3,3,256, n_boxes["conv10_2"] * n_classes, ws=1,  pad=1)
    cl_conv11_2 = Conv(3,3,256, n_boxes["conv11_2"] * n_classes, ws=1, pad=1)
    
    
     # Predict localization boxes' bounds (as offsets w.r.t prior-boxes)  4 offsets: ∆cx, ∆cy, w and h
    l_conv4_3 = loc_conv4_3(conv4_3_feats)  # (38, 38,16)
    l_conv4_3 = reshape(l_conv4_3,  expectedShape(length(l_conv4_3),batch_size,4) )
    
    
    l_conv7 = loc_conv7(conv7_feats)  # (19, 19, 24,1)
    l_conv7 = reshape(l_conv7,  expectedShape(length(l_conv7),batch_size,4) )
    
    l_conv8_2 = loc_conv8_2(conv8_2_feats)  # (10, 10, 24, 1)
    l_conv8_2 = reshape(l_conv8_2, expectedShape(length(l_conv8_2),batch_size,4)  )
    
    l_conv9_2 = loc_conv9_2(conv9_2_feats)  # (5, 5, 24, 1)
    l_conv9_2 = reshape(l_conv9_2, expectedShape(length(l_conv9_2),batch_size,4) )
    
    l_conv10_2 = loc_conv10_2(conv10_2_feats)  # (3, 3, 16, 1)
    l_conv10_2 = reshape(l_conv10_2, expectedShape(length(l_conv10_2),batch_size,4) )
    
    
    l_conv11_2 = loc_conv11_2(conv11_2_feats)  # (1, 1, 16, 1)
    l_conv11_2 = reshape(l_conv11_2, expectedShape(length(l_conv11_2),batch_size,4) )
    
    
    
    # Predict classes in localization boxes
    c_conv4_3 = cl_conv4_3(conv4_3_feats)  # (38, 38, 4 * n_classes, 1) --> (38, 38, 84, 1)
    c_conv4_3 = reshape(c_conv4_3, expectedShape(length(c_conv4_3),batch_size,n_classes) )
    
    c_conv7 = cl_conv7(conv7_feats)         # (19, 19, 6 * n_classes,1) -->  (19, 19, 126,1)
    c_conv7 = reshape(c_conv7, expectedShape(length(c_conv7),batch_size,n_classes) )
    
    c_conv8_2 = cl_conv8_2(conv8_2_feats)  # (10, 10, 6 * n_classes, 1) --> (10,10,24,1)
    c_conv8_2 = reshape(c_conv8_2, expectedShape(length(c_conv8_2),batch_size,n_classes) )
    
    c_conv9_2 = cl_conv9_2(conv9_2_feats)  # (5, 5, 6 * n_classes, 1)   --> (5,5,24,1)
    c_conv9_2 = reshape(c_conv9_2, expectedShape(length(c_conv9_2),batch_size,n_classes) )
        
    c_conv10_2 = cl_conv10_2(conv10_2_feats)  # ( 3, 3, 4 * n_classes,1)-->  (5,5,16,1)
    c_conv10_2 = reshape(c_conv10_2, expectedShape(length(c_conv10_2),batch_size,n_classes) )
    
    
    c_conv11_2 = cl_conv11_2(conv11_2_feats)  # ( 1, 1,4 * n_classes,1) --> (1,1,16,1)
    c_conv11_2 = reshape(c_conv11_2, expectedShape(length(c_conv11_2),batch_size,n_classes) )
    
    println("l_conv4_3:",size(l_conv4_3))
    println("c_conv4_3:",size(c_conv4_3))
    println()
    
    println("l_conv7:",size(l_conv7))
    println("c_conv7:",size(c_conv7))
    println()
    
    println("l_conv8_2:",size(l_conv8_2))
    println("c_conv8_2:",size(c_conv8_2))
    
    println()
    
    println("l_conv9_2:",size(l_conv9_2))
    println("c_conv9_2:",size(c_conv9_2))
    
    println()
    
    
    println("l_conv10_2:",size(l_conv10_2))
    println("c_conv10_2:",size(c_conv10_2))
    println()
    
    println("l_conv11_2:",size(l_conv11_2))
    println("c_conv11_2:",size(c_conv11_2))
    
    println()
    
    
    
    locs = vcat(l_conv4_3, l_conv7, l_conv8_2, l_conv9_2, l_conv10_2, l_conv11_2)  # (8732, 4,1)
    classes_scores = vcat(c_conv4_3, c_conv7, c_conv8_2, c_conv9_2, c_conv10_2, c_conv11_2
                                   )  # (8732, n_classes,1)
    
    return locs,classes_scores
end
    

    

boundingBox_Location_Class_Predictions (generic function with 1 method)

In [217]:
#@doc vcat

In [214]:
locs,classes_scores = boundingBox_Location_Class_Predictions(x,vgg16)

Batch size : 1
KnetArray{Float32,4}
Size conv4_3_feats : (38, 38, 512, 1)
Size conv7_feats : (19, 19, 1024, 1)
l_conv4_3:(5776, 4, 1)
c_conv4_3:(5776, 21, 1)

l_conv7:(2166, 4, 1)
c_conv7:(2166, 21, 1)

l_conv8_2:(600, 4, 1)
c_conv8_2:(600, 21, 1)

l_conv9_2:(150, 4, 1)
c_conv9_2:(150, 21, 1)

l_conv10_2:(36, 4, 1)
c_conv10_2:(36, 21, 1)

l_conv11_2:(4, 4, 1)
c_conv11_2:(4, 21, 1)



(K32(8732,4,1)[0.043867495⋯], K32(8732,21,1)[0.05549546⋯])

In [215]:
size(locs)

(8732, 4, 1)

In [216]:
size(classes_scores)

(8732, 21, 1)

In [251]:
fmap_dims = OrderedDict(
    "conv4_3" => 38,
    "conv7" => 19,
    "conv8_2" => 10,
    "conv9_2" => 5,
    "conv10_2" => 3,
    "conv11_2" => 1)

#Scales calculated as smin + ((Smax -Smin)/ m-1)*(k-1) where smin = 0.2 smax = 0.9 
#k in the range [1,m] where m-> number of feature maps
#
obj_scales = OrderedDict(
    "conv4_3" => 0.1,
    "conv7" => 0.2,
    "conv8_2" => 0.375,
    "conv9_2" => 0.55,
    "conv10_2" => 0.725,
    "conv11_2" => 0.9)

#Predefined by authors
aspect_ratios = OrderedDict(
    "conv4_3" => [1., 2., 0.5],
    "conv7" => [1., 2., 3., 0.5, .333],
    "conv8_2" => [1., 2., 3., 0.5, .333],
    "conv9_2" => [1., 2., 3., 0.5, .333],
    "conv10_2" => [1., 2., 0.5],
    "conv11_2" => [1., 2., 0.5])


OrderedDict{String,Array{Float64,1}} with 6 entries:
  "conv4_3"  => [1.0, 2.0, 0.5]
  "conv7"    => [1.0, 2.0, 3.0, 0.5, 0.333]
  "conv8_2"  => [1.0, 2.0, 3.0, 0.5, 0.333]
  "conv9_2"  => [1.0, 2.0, 3.0, 0.5, 0.333]
  "conv10_2" => [1.0, 2.0, 0.5]
  "conv11_2" => [1.0, 2.0, 0.5]

In [273]:
function create_prior_boxes()
    fmaps =keys(fmap_dims)
    prior_boxes = []
    additional_scale = 1
    for (k, fmap) in enumerate(fmaps) #"conv4_3" conv7""conv8_2" ,"conv9_2" "conv10_2" "conv11_2"
        for i in 1:fmap_dims[fmap]
            for j in 1:fmap_dims[fmap]
                cx = (j + 0.5) / fmap_dims[fmap]
                cy = (i + 0.5) / fmap_dims[fmap]
                
                for ratio in aspect_ratios[fmap]
                    push!(prior_boxes, [cx, cy, obj_scales[fmap] * sqrt(ratio), obj_scales[fmap] / sqrt(ratio)])
                    
                    if ratio==1
                        try
                        additional_scale = sqrt(obj_scales[fmap] * obj_scales[fmaps[k + 1]])
                        catch
                            additional_scale = 1
                        end
                        push!(prior_boxes,[cx, cy, additional_scale, additional_scale])
                    end
                end
            end
        end
    end
    #prior_boxes = KnetArray(prior_boxes) # (8732, 4)
    prior_boxes = KnetArray(permutedims(hcat(prior_boxes...),(2,1)))
    return prior_boxes                               
    
end

create_prior_boxes (generic function with 1 method)

In [274]:
pb = create_prior_boxes()

8732×4 KnetArray{Float64,2}:
 0.0394737  0.0394737  0.1        0.1
 0.0394737  0.0394737  1.0        1.0
 0.0394737  0.0394737  0.141421   0.0707107
 0.0394737  0.0394737  0.0707107  0.141421
 0.0657895  0.0394737  0.1        0.1
 0.0657895  0.0394737  1.0        1.0
 0.0657895  0.0394737  0.141421   0.0707107
 0.0657895  0.0394737  0.0707107  0.141421
 0.0921053  0.0394737  0.1        0.1
 0.0921053  0.0394737  1.0        1.0
 0.0921053  0.0394737  0.141421   0.0707107
 0.0921053  0.0394737  0.0707107  0.141421
 0.118421   0.0394737  0.1        0.1
 ⋮                                
 0.833333   1.16667    0.725      0.725
 0.833333   1.16667    1.0        1.0
 0.833333   1.16667    1.0253     0.512652
 0.833333   1.16667    0.512652   1.0253
 1.16667    1.16667    0.725      0.725
 1.16667    1.16667    1.0        1.0
 1.16667    1.16667    1.0253     0.512652
 1.16667    1.16667    0.512652   1.0253
 1.5        1.5        0.9        0.9
 1.5        1.5        1.0        1.0
 1.5     

In [282]:
function KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Cptr, pointer(x))
    k = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end

KnetArray

In [ ]:
# Next work ---> Matching 
function detect_object( )
   
    
    
end